In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
# import tensorflow.keras as tfkeras

from tensorflow.keras.models import Sequential              
from tensorflow.keras.layers import Dense                   
from tensorflow.keras.optimizers import SGD                 # 최적화 모델
from tensorflow.keras.callbacks import EarlyStopping        # 모델 사전 종료 라이브러리 (모델 분석 최적화를 위함)

# K-fold Model 호출
from sklearn.model_selection import train_test_split, KFold
# 데이터 정규화 라이브러리
from sklearn.preprocessing import StandardScaler

2025-04-13 21:08:37.495779: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-13 21:08:37.496375: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-13 21:08:37.499832: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-13 21:08:37.507476: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744546117.520411 1329246 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744546117.52

In [2]:
#파일 불러오기 및 데이터 처리
df = pd.read_csv('sleep_deprivation_dataset_detailed.csv')

# 불필요 데이터 제외
df = df.drop(columns=['Participant_ID', 'Stroop_Task_Reaction_Time'])
#print(df.head())

In [3]:
# Preprocess

# 기대예측값 분리
y_data = df['PVT_Reaction_Time']

# 입력값에서 기대예측값 제거
df = df.drop(columns=['PVT_Reaction_Time'])

# Gender를 one-hot encoding해서 bool을 전부 float로 casting하기
df = pd.get_dummies(df, columns=['Gender']).astype(float)

# 이제 저 dataframe을 입력에 저장
x_data = df

In [4]:
#데이터 분할(train, val, test)
x_train, x_s30, y_train, y_s30 =    train_test_split(x_data, y_data, test_size=0.3, random_state=42)
x_val, x_test, y_val, y_test =      train_test_split(x_s30, y_s30, test_size=0.5, random_state=42)

In [5]:
# 데이터 정규화 객체 만들기
# x_train(즉 입력값)용으로 만들어졌으니 재사용이 가능하다.
scaler = StandardScaler()

# 최초정규화를 위해 x_train 순회 돌기
x_train = scaler.fit_transform(x_train)

# 정규화: x_val
x_val = scaler.transform(x_val)

# 정규화: x_test
x_test = scaler.transform(x_test)

In [6]:
# 선형모델
model = Sequential()

# 모델의 출력층 정의
model.add(Dense(
    1               # 출력층: 1
    , input_dim=12  # 입력층: 12
    )
)

# 모델 구조 출력
model.summary()

# 옵티마이저 미리 설정
sgd_optimizer = SGD(
    learning_rate=0.0092
    )

/home/kimdowne/myfile/ML/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2025-04-13 21:08:39.180358: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 1)              │            13 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 13 (52.00 B)

 Trainable params: 13 (52.00 B)

 Non-trainable params: 0 (0.00 B)

In [7]:
# EarlyStopping, patience회 적발 시 스탑
early_stopping = EarlyStopping(
    monitor='val_loss'              # loss출력값 확인
    , patience=30                   # 30번을 카운트로
    , restore_best_weights=True     # 가중치 예쁘게 나온 거 저장
    )

In [8]:
# 모델 컴파일
model.compile(
    loss='mse'                  # mse loss
    , optimizer=sgd_optimizer   # 미리 만든 sgd_optimizer를 사용
    , metrics=['mae']           # mae로 loss값 확인
    ) 

In [9]:
#모델 학습
history = model.fit(
    x_train                             # 입력값
    , y_train                           # 기대대응출력값
    , epochs=200                        # 반복
    , batch_size=8                      # 업데이트를 위한 비교수
    , validation_data=(x_val, y_val)    # 검증
    , callbacks=[early_stopping]        # early_stopping를 스텝마다 소환
    ) 

Epoch 1/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 120481.2969 - mae: 337.7953 - val_loss: 69920.4688 - val_mae: 250.9467
Epoch 2/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 95008.8516 - mae: 297.5187 - val_loss: 52426.5820 - val_mae: 213.2222
Epoch 3/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 78868.5859 - mae: 268.6171 - val_loss: 38556.1094 - val_mae: 178.5018
Epoch 4/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 61693.0664 - mae: 236.1235 - val_loss: 29616.2422 - val_mae: 151.5947
Epoch 5/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 51493.7461 - mae: 214.0987 - val_loss: 23238.8184 - val_mae: 130.5774
Epoch 6/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 39532.9414 - mae: 182.8599 - val_loss: 18214.9746 - val_mae: 110.2302
Epoch 7/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 35801.5430 - mae: 175.1835 - val_loss: 14560.5068 - val_mae: 92.9530
Epoch 8/200
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 25657.9336 - mae: 144.6167 - val_loss: 11551.9

In [10]:
# Test 평가
test_loss, test_mae = model.evaluate(x_test, y_test)
print(f'Test Loss: {test_loss}, Test MAE: {test_mae}')

# 예측 값
predictions = model.predict(x_test)
print(predictions[:5])

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 18289.0273 - mae: 95.8601
Test Loss: 18289.02734375, Test MAE: 95.86009216308594
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
[[299.97012]
 [265.24847]
 [206.90414]
 [217.41342]
 [320.41202]]


In [ ]:
# K-Fold ================================================================

# 데이터 분리
x_temp, x_test, y_temp, y_test = train_test_split(
    x_data
    , y_data
    , test_size=0.15
    , random_state=42
    )

# 입력값에서 테스트용 입력 분리하기
x_train_kfold, x_val_holdout, y_train_kfold, y_val_holdout = train_test_split(
    x_temp
    , y_temp
    , test_size=0.1765
    , random_state=4
    )

In [ ]:
# 이미 만들어진 scaler transform이 있으니 새로 생성할 필요 없이 그대로 반영
x_train_kfold = scaler.transform(x_train_kfold)
x_val_holdout = scaler.transform(x_val_holdout)
x_test = scaler.transform(x_test)

In [13]:
# K-fold 기본 설정, 5분할, 섞기O
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# MAE 저장 리스트
mae_history = []

In [14]:
#모델
model_kfold = Sequential()

#모델 층 추가
model_kfold.add(
    Dense(
        1                               # 출력층 1
        , input_dim=x_train.shape[1]    # 입력층: x_train의 열 수로 설정
        )
    )

/home/kimdowne/myfile/ML/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [15]:
# k-fold (기본 k-fold 설정에 기반해 각 )
for train_idx, val_idx in kfold.split(x_train_kfold):
    x_train, x_val = x_train_kfold[train_idx], x_train_kfold[val_idx]

    # pandas index을 사용 (iloc: )
    y_train, y_val = y_train_kfold.iloc[train_idx], y_train_kfold.iloc[val_idx]  

    # 모델을 매번 만드는 것도 어려우니, keras에서 제공하는 모델을 클론해 보자
    _model = tf.keras.models.clone_model(model_kfold)

    # 옵티마이저 정의
    sgd_optimizer = SGD(learning_rate=0.0092)

    #모델 컴파일
    _model.compile(
        loss='mse'
        , optimizer=sgd_optimizer
        , metrics=['mae']
        )

    # 모델학습
    _model.fit(
        x_train         # 입력값 
        , y_train       # 상응기대값
        , epochs=200    # 200번 반복
        , batch_size=8  # 비교 8회마다 업데이트
        , verbose=0     # 모델 학습 도중 뭐 하나 말하지 말라는 거
        , validation_data=(x_val, y_val)
        )

    #검증 데이터 평가 돌리기
    _, val_mae = _model.evaluate(x_val, y_val, verbose=0)

    # 나중에 출력하려고 추가하는 거
    mae_history.append(val_mae)

In [16]:
# K-Fold MAE 출력
print("각 분할 MAE: ", mae_history)
print(f"K-Fold 평균 MAE:  {np.mean(mae_history):.4f}")

각 분할 MAE:  [83.78390502929688, 91.146728515625, 79.07701873779297, 65.15189361572266, 62.864593505859375]
K-Fold 평균 MAE:  76.4048


In [17]:
sgd_optimizer = SGD(learning_rate=0.0092)

#모델 컴파일
model_kfold.compile(
    loss='mse'
    , optimizer=sgd_optimizer
    , metrics=['mae']
    )

In [18]:
# 모델학습
model_kfold.fit(
    x_train         # 입력값 
    , y_train       # 상응기대값
    , epochs=200    # 200번 반복
    , batch_size=8  # 비교 8회마다 업데이트
    , verbose=0     # 모델 학습 도중 뭐 하나 말하지 말라는 거
    , validation_data=(x_val, y_val)
    , callbacks=[early_stopping]        # early_stopping를 스텝마다 소환
    )

In [19]:
# 두 모델에 대해서 성능 테스트해 보기 (kfold와 기존 model 변경)
for name, mod in [("model", model), ("model_kfold", model_kfold)]:
    print(f"성능 확인: {name}")

    # 성능 확인
    val_loss, val_mae = mod.evaluate(x_val_holdout, y_val_holdout, verbose=0)
    print(f"검증 세트 MAE: {val_mae:.4f}")

    # 최종? 모델 평가
    test_loss, test_mae = mod.evaluate(x_test, y_test, verbose=0)
    print(f"평가 세트 MAE (test): {test_mae:.4f}")
    print()

    pass

성능 확인: model
검증 세트 MAE: 63.9663
평가 세트 MAE (test): 107.2981

성능 확인: model_kfold
검증 세트 MAE: 75.6628
평가 세트 MAE (test): 126.4120

